In [26]:
import pandas as pd

In [27]:
df = pd.read_csv('user_reviews.csv')

In [28]:
df.head()

,Unnamed: 0,user_id,user_url,reviews
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [29]:
df['reviews'][0]

'[{\'funny\': \'\', \'posted\': \'Posted November 5, 2011.\', \'last_edited\': \'\', \'item_id\': \'1250\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.\'}, {\'funny\': \'\', \'posted\': \'Posted July 15, 2011.\', \'last_edited\': \'\', \'item_id\': \'22200\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': "It\'s unique and worth a playthrough."}, {\'funny\': \'\', \'posted\': \'Posted April 21, 2011.\', \'last_edited\': \'\', \'item_id\': \'43110\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!\'}

In [30]:
df = df.drop(columns="Unnamed: 0")

In [31]:
df.shape

(25799, 3)

In [32]:
import ast
def parse_review(reviews):
        return ast.literal_eval(reviews)
    
df['reviews'] = df['reviews'].apply(parse_review)

In [33]:
df['reviews'][1]

[{'funny': '',
  'posted': 'Posted June 24, 2014.',
  'last_edited': '',
  'item_id': '251610',
  'helpful': '15 of 20 people (75%) found this review helpful',
  'recommend': True,
  'review': 'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8'},
 {'funny': '',
  'posted': 'Posted September 8, 2013.',
  'last_edited': '',
  'item_id': '227300',
  'helpful': '0 of 1 people (0%) found this review helpful',
  'recommend': True,
  'review': "For a simple (it's actually not all th

In [129]:
# Reduzco la cantidad de filas porque sino mi compu explota
df = df.head(5000)

# Endpoints

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True

In [130]:
import re

def normalize_item_id(item_id):
    # Convertir item_id a cadena y luego eliminar caracteres especiales y convertir a minúsculas
    item_id = re.sub(r'[^\w\s]', '', str(item_id)).lower()
    return item_id

steam_games = pd.read_csv('cleaned_data.csv')
data_list = []  # Lista para almacenar los datos

# Iterar a través de las reviews
for index, row in df.iterrows():
    for review in row['reviews']:
        item_id = review.get('item_id', '')
        posted = review.get('posted', '')
        review_year = review.get('posted', '').split()[-1].strip('.')
        recommend = review.get('recommend', False)
        
            
        if recommend:
            normalized_item_id = normalize_item_id(item_id)
            # Verificar si el juego existe en steam_games después de normalizar el item_id
            if normalized_item_id in steam_games['id'].apply(normalize_item_id).values:
                developer = steam_games[steam_games['id'].apply(normalize_item_id) == normalized_item_id]['developer'].values[0]
                data_list.append({'year': review_year, 'developer': developer})
            else:
                # Manejo de casos en los que no se encuentra el juego
                developer = 'Desarrollador Desconocido'
                data_list.append({'year': review_year, 'developer': developer})
    
    
data = pd.DataFrame(data_list)  # Convertir la lista en un DataFrame

In [134]:
# Crear un DataFrame vacío para almacenar los resultados
result_df = pd.DataFrame(columns=['year', 'developer', 'count'])

# Iterar desde el año 2005 al 2020
for year in range(2005, 2021):
    developers_for_year = data[data['year'] == str(year)]['developer']
    # Filtrar los desarrolladores desconocidos
    developers_for_year = developers_for_year[developers_for_year != 'Desarrollador Desconocido']
    top_developers = developers_for_year.value_counts().reset_index()
    top_developers.columns = ['developer', 'count']
    top_developers = top_developers.head(3)
    top_developers['year'] = year

    result_df = pd.concat([result_df, top_developers])

# Reiniciar los índices del DataFrame resultante
result_df = result_df.reset_index(drop=True)

print(result_df)

    year             developer count
0   2010                 Valve     3
1   2010  Tripwire Interactive     1
2   2010      Frictional Games     1
3   2011                 Valve     8
4   2011      Supergiant Games     2
5   2011  Tripwire Interactive     1
6   2012                 Valve    18
7   2012        Telltale Games     2
8   2012     Facepunch Studios     2
9   2013                 Valve    44
10  2013     Facepunch Studios     9
11  2013           Chucklefish     6
12  2014                 Valve   107
13  2014     Facepunch Studios    36
14  2014   Bohemia Interactive    19
15  2015                 Valve   106
16  2015     Facepunch Studios    24
17  2015   Bohemia Interactive    17


In [ ]:
#Almaceno el df en un csv para ser consumido por la API.
result_df.to_csv("df_endpoint4.csv", index=False)

In [136]:
def best_developer_year(year, result_df):
    # Filtrar el DataFrame por el año especificado
    filtered_data = result_df[result_df['year'] == year]

    # Ordenar los datos por la columna 'count' de forma descendente
    sorted_data = filtered_data.sort_values(by='count', ascending=False)

    # Obtener el top 3 de desarrolladores
    top_developers = sorted_data.head(3)

    # Crear la lista en el formato deseado
    top_developers_list = [{"Puesto 1": top_developers.iloc[0]['developer']}, 
                           {"Puesto 2": top_developers.iloc[1]['developer']},
                           {"Puesto 3": top_developers.iloc[2]['developer']}]

    return top_developers_list

# Llama a la función con el año deseado y el DataFrame result_df
year = 2015  # Cambia esto al año que desees
top_developers_2015 = best_developer_year(year, result_df)

# Imprime el resultado
print(top_developers_2015)

[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Facepunch Studios'}, {'Puesto 3': 'Bohemia Interactive'}]


#### Endpoint sentiment analysis

* def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [137]:
df_reviews = pd.DataFrame(df["reviews"])

In [139]:
from textblob import TextBlob
def analyze_sentiment(reviews_list):
    sentiments = []
    for review_dict in reviews_list:
        analysis = TextBlob(review_dict['review'])
        polarity = analysis.sentiment.polarity
        if polarity < 0:
            sentiments.append(0) 
        elif polarity == 0:
            sentiments.append(1)  
        else:
            sentiments.append(2) 
    return sentiments

In [ ]:
df_reviews['sentiment_analysis'] = df_reviews['reviews'].apply(analyze_sentiment)

In [ ]:
df_reviews.iloc[1]

* def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [91]:
df["reviews"][4]

[{'funny': '3 people found this review funny',
  'posted': 'Posted April 15, 2014.',
  'last_edited': '',
  'item_id': '211420',
  'helpful': '35 of 43 people (81%) found this review helpful',
  'recommend': True,
  'review': 'Git gud'},
 {'funny': '1 person found this review funny',
  'posted': 'Posted December 23, 2013.',
  'last_edited': '',
  'item_id': '211820',
  'helpful': '12 of 16 people (75%) found this review helpful',
  'recommend': True,
  'review': "It's like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."},
 {'funny': '2 people found this review funny',
  'posted': 'Posted March 14, 2014.',
  'last_edited': '',
  'item_id': '730',
  'helpful': '5 of 5 people (100%) found this review helpful',
  'recommend': True,
  'review': 'Hold shift to win, Hold CTRL to lose.'},
 {'funny': '',
  'posted': 'Posted July 11, 2013.',
  'last_edited': '',
  'item_id': '204300',
  'helpful': 'No ratings yet',
  'recommend': True,
  'rev

In [81]:
df_reviews = pd.DataFrame(df["reviews"])

In [82]:
from textblob import TextBlob
def analyze_sentiment(reviews_list):
    sentiments = []
    for review_dict in reviews_list:
        analysis = TextBlob(review_dict['review'])
        polarity = analysis.sentiment.polarity
        if polarity < 0:
            sentiments.append(0) 
        elif polarity == 0:
            sentiments.append(1)  
        else:
            sentiments.append(2) 
    return sentiments

In [83]:
df_reviews['sentiment_analysis'] = df_reviews['reviews'].apply(analyze_sentiment)

In [87]:
df_reviews.iloc[1]

reviews               [{'funny': '', 'posted': 'Posted June 24, 2014...
sentiment_analysis                                            [2, 0, 0]
Name: 1, dtype: object

In [201]:
df_reviews["reviews"][37]

[{'funny': '',
  'posted': 'Posted January 6, 2015.',
  'last_edited': '',
  'item_id': '213670',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'My character is a Jew named ♥♥♥♥♥♥♥♥♥ that is wearing a KKK costume.10/10, IGN'}]

In [103]:
import re
def sentiment_analysis(year):
    df_filtered = df_reviews[df_reviews['reviews'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

    pattern = r'Posted (\w+ \d{1,2}, \d{4}).'
    def extract_year(review):
        match = re.search(pattern, review[0]['posted'])
        if match:
            return int(match.group(1).split()[-1])
        else:
            return None

    df_filtered['posted_year'] = df_filtered['reviews'].apply(extract_year)

    filtered_reviews = df_filtered[df_filtered['posted_year'] == year]

    sentiment_counts = filtered_reviews['sentiment_analysis'].explode().value_counts().to_dict()

    sentiment_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    sentiment_counts = {sentiment_labels[key]: value for key, value in sentiment_counts.items()}

    return sentiment_counts

In [110]:
years_of_interest = list(range(2009, 2016))
results_dict = {'Year': [], 'Sentiment Analysis': []}

for year in years_of_interest:
    sentiment_result = sentiment_analysis(year)
    results_dict['Year'].append(year)
    results_dict['Sentiment Analysis'].append(sentiment_result)

results_df = pd.DataFrame(results_dict)

C:\Users\jazmin\AppData\Local\Temp\ipykernel_14092\779080910.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['posted_year'] = df_filtered['reviews'].apply(extract_year)
C:\Users\jazmin\AppData\Local\Temp\ipykernel_14092\779080910.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['posted_year'] = df_filtered['reviews'].apply(extract_year)
C:\Users\jazmin\AppData\Local\Temp\ipykernel_14092\779080910.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [112]:
results_df.to_csv("df_endpoint6.csv",index=False)

In [113]:
results_df

,Year,Sentiment Analysis
0,2009,{}
1,2010,"{'Positive': 15, 'Negative': 5, 'Neutral': 1}"
2,2011,"{'Positive': 164, 'Negative': 50, 'Neutral': 35}"
3,2012,"{'Positive': 423, 'Negative': 119, 'Neutral': 80}"
4,2013,"{'Positive': 3198, 'Neutral': 1149, 'Negative'..."
5,2014,"{'Positive': 12493, 'Neutral': 4701, 'Negative..."
6,2015,"{'Positive': 10757, 'Negative': 4426, 'Neutral..."
